# Deep Hedging sous Black-Scholes avec Sauts de Merton

## Introduction

Ce notebook implémente une stratégie de **Deep Hedging** pour une option européenne dans un monde de type **Black-Scholes** avec possibilité de **sauts de Merton**.

### Objectifs

1. Comparer une stratégie de couverture classique (Delta Hedging) à une stratégie apprise via réseaux de neurones
2. Optimiser la couverture en minimisant le risque de queue (CVaR)
3. Gérer les frictions de marché (coûts de transaction, rebalancement discret)

### Architecture

- **Simulateur**: Trajectoires Black-Scholes ou Merton
- **Modèle neuronal**: MLP ou LSTM générant des ratios de couverture
- **Fonction de perte**: CVaR (Conditional Value-at-Risk)
- **Comparaison**: Delta Hedging analytique vs Deep Hedging

## 1. Configuration et Imports

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, Tuple, Optional

# Configuration du device
DTYPE = torch.float32

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")

print(f"Device: {DEVICE}, dtype: {DTYPE}")

# Style des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 2. Configuration des Paramètres

Définition des paramètres de marché, de training et globaux via `dataclasses`.

In [ ]:
@dataclass
class MarketConfig:
    """Configuration des paramètres de marché."""
    # Black-Scholes
    S0: float = 100.0          # Prix initial
    r: float = 0.03            # Taux sans risque
    q: float = 0.0             # Dividende
    sigma: float = 0.2         # Volatilité
    T: float = 1.0             # Maturité
    n_steps: int = 52          # Nombre de steps de hedging
    
    # Option
    K: float = 100.0           # Strike
    is_call: bool = True       # Call ou Put
    
    # Merton Jump-Diffusion
    use_jumps: bool = False    # Activer les sauts
    lambda_jump: float = 1.0   # Intensité de Poisson
    mu_J: float = -0.1         # Moyenne du log-jump
    sigma_J: float = 0.3       # Std du log-jump
    
    # Frictions
    cost_s: float = 0.0002     # Coûts de transaction
    
    # Données
    n_paths_train: int = 200_000
    n_paths_val: int = 50_000
    
    def __post_init__(self):
        """Validation des paramètres."""
        assert self.T > 0, "T must be positive"
        assert self.sigma > 0, "sigma must be positive"
        assert self.S0 > 0, "S0 must be positive"
        assert self.K > 0, "K must be positive"
        assert self.n_steps > 0, "n_steps must be positive"

@dataclass
class TrainingConfig:
    """Configuration du training."""
    n_epochs: int = 50
    batch_size: int = 10_000
    lr: float = 1e-3
    cvar_alpha: float = 0.025   # Niveau de CVaR
    print_every: int = 5
    
    def __post_init__(self):
        assert 0 < self.cvar_alpha < 1, "cvar_alpha must be in (0,1)"

@dataclass
class DeepHedgingConfig:
    """Configuration globale."""
    market: MarketConfig = MarketConfig()
    training: TrainingConfig = TrainingConfig()
    device: torch.device = DEVICE
    dtype: torch.dtype = DTYPE
    seed: int = 42

# Instance globale
cfg = DeepHedgingConfig()
print(f"Configuration créée: {cfg.market.n_steps} steps, CVaR α={cfg.training.cvar_alpha}")

## 3. Simulation des Prix (Black-Scholes et Merton)

Implémentation des simulateurs de trajectoires avec **vectorisation optimale**.

In [ ]:
class BlackScholesWorld:
    """Simulateur Black-Scholes optimisé."""
    
    def __init__(self, config: MarketConfig):
        self.cfg = config
        self.dt = config.T / config.n_steps
        self.t_grid = np.linspace(0, config.T, config.n_steps + 1)
        
    def simulate_paths(self, n_paths: int, seed: int = 42) -> Dict[str, np.ndarray]:
        """
        Simule des trajectoires BS (version vectorisée optimale).
        
        Returns:
            dict avec 'S', 'dS', 'payoff', 't_grid'
        """
        rng = np.random.default_rng(seed)
        m = self.cfg
        
        # Bruits gaussiens
        Z = rng.standard_normal((n_paths, m.n_steps)).astype(np.float32)
        
        # Drift et diffusion
        drift = (m.r - m.q - 0.5 * m.sigma**2) * self.dt
        vol = m.sigma * np.sqrt(self.dt)
        
        # Simulation vectorisée complète
        log_returns = drift + vol * Z
        log_S = np.cumsum(log_returns, axis=1)
        
        S = np.zeros((n_paths, m.n_steps + 1), dtype=np.float32)
        S[:, 0] = m.S0
        S[:, 1:] = m.S0 * np.exp(log_S)
        
        # Incréments
        dS = np.diff(S, axis=1)
        
        # Payoff
        if m.is_call:
            payoff = np.maximum(S[:, -1] - m.K, 0.0)
        else:
            payoff = np.maximum(m.K - S[:, -1], 0.0)
        
        return {
            'S': S,
            'dS': dS,
            'payoff': payoff,
            't_grid': self.t_grid
        }

class MertonWorld(BlackScholesWorld):
    """Simulateur Merton Jump-Diffusion."""
    
    def __init__(self, config: MarketConfig):
        super().__init__(config)
        # Correction du drift pour risque neutre
        self.kappa = np.exp(config.mu_J + 0.5 * config.sigma_J**2) - 1.0
        
    def simulate_paths(self, n_paths: int, seed: int = 42) -> Dict[str, np.ndarray]:
        """Simule des trajectoires Merton avec sauts."""
        rng = np.random.default_rng(seed)
        m = self.cfg
        
        S = np.zeros((n_paths, m.n_steps + 1), dtype=np.float64)
        S[:, 0] = m.S0
        
        drift = (m.r - m.q - m.lambda_jump * self.kappa - 0.5 * m.sigma**2) * self.dt
        vol = m.sigma * np.sqrt(self.dt)
        
        for t in range(m.n_steps):
            # Diffusion
            Z = rng.standard_normal(n_paths)
            
            # Sauts
            N_jump = rng.poisson(m.lambda_jump * self.dt, n_paths)
            log_jump = np.zeros(n_paths, dtype=np.float64)
            
            has_jump = N_jump > 0
            if np.any(has_jump):
                n_total_jumps = N_jump[has_jump].sum()
                Y = rng.normal(m.mu_J, m.sigma_J, n_total_jumps)
                
                idx = np.concatenate([[0], np.cumsum(N_jump[has_jump])])
                for k in range(len(idx) - 1):
                    log_jump[np.where(has_jump)[0][k]] = Y[idx[k]:idx[k+1]].sum()
            
            # Mise à jour
            dlog_S = drift + vol * Z + log_jump
            S[:, t+1] = S[:, t] * np.exp(dlog_S)
        
        dS = np.diff(S, axis=1)
        
        if m.is_call:
            payoff = np.maximum(S[:, -1] - m.K, 0.0)
        else:
            payoff = np.maximum(m.K - S[:, -1], 0.0)
        
        return {
            'S': S.astype(np.float32),
            'dS': dS.astype(np.float32),
            'payoff': payoff.astype(np.float32),
            't_grid': self.t_grid
        }

# Test rapide
world = BlackScholesWorld(cfg.market)
test_data = world.simulate_paths(1000, seed=42)
print(f"Shapes: S={test_data['S'].shape}, dS={test_data['dS'].shape}, payoff={test_data['payoff'].shape}")
print(f"Prix moyen final: {test_data['S'][:, -1].mean():.2f}")
print(f"Payoff moyen: {test_data['payoff'].mean():.4f}")

## 4. Modèle Neuronal de Couverture

Architecture MLP générant les ratios de hedging à partir de l'état du marché.

In [ ]:
class HedgingNetwork(nn.Module):
    """Réseau de neurones pour la stratégie de hedging."""
    
    def __init__(self, hidden_sizes=[64, 64], activation='relu'):
        super().__init__()
        self.hidden_sizes = hidden_sizes
        
        # Input: [S_normalized, time_to_maturity]
        layers = []
        in_dim = 2
        
        for h_dim in hidden_sizes:
            layers.append(nn.Linear(in_dim, h_dim))
            if activation == 'relu':
                layers.append(nn.ReLU())
            elif activation == 'tanh':
                layers.append(nn.Tanh())
            in_dim = h_dim
        
        # Output: hedge ratio
        layers.append(nn.Linear(in_dim, 1))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, S, t, S0, T):
        """
        Args:
            S: Prix du sous-jacent (batch_size,)
            t: Temps actuel (batch_size,)
            S0: Prix initial pour normalisation
            T: Maturité pour normalisation
        
        Returns:
            Hedge ratio (batch_size,)
        """
        S_norm = S / S0
        tau = 1.0 - t / T  # Time to maturity
        
        x = torch.stack([S_norm, tau], dim=-1)
        hedge_ratio = self.network(x).squeeze(-1)
        
        return hedge_ratio

# Test du modèle
model = HedgingNetwork(hidden_sizes=[32, 32]).to(DEVICE)
print(f"Modèle créé: {sum(p.numel() for p in model.parameters())} paramètres")

# Test forward pass
S_test = torch.tensor([100.0, 105.0, 95.0], device=DEVICE)
t_test = torch.tensor([0.0, 0.5, 1.0], device=DEVICE)
ratios = model(S_test, t_test, cfg.market.S0, cfg.market.T)
print(f"Hedge ratios de test: {ratios}")

## 5. Fonction de Perte CVaR

Implémentation de la Conditional Value-at-Risk pour optimiser le risque de queue.

In [ ]:
class CVaRLoss(nn.Module):
    """Fonction de perte CVaR (Rockafellar & Uryasev)."""
    
    def __init__(self, alpha=0.025):
        super().__init__()
        self.alpha = alpha
        
    def forward(self, pnl):
        """
        Calcule le CVaR du PnL.
        
        Args:
            pnl: Profit & Loss (batch_size,)
        
        Returns:
            CVaR (scalaire)
        """
        # On minimise -PnL, donc on cherche le CVaR des pertes
        losses = -pnl
        
        # VaR: quantile à alpha
        var = torch.quantile(losses, self.alpha)
        
        # CVaR: moyenne des pertes au-delà du VaR
        excess_losses = torch.relu(losses - var)
        cvar = var + excess_losses.mean() / self.alpha
        
        return cvar

# Test de la loss
cvar_loss = CVaRLoss(alpha=0.025)
pnl_test = torch.randn(1000) * 10  # PnL simulé
loss_val = cvar_loss(pnl_test)
print(f"CVaR test: {loss_val:.4f}")
print(f"Moyenne PnL: {pnl_test.mean():.4f}, Std: {pnl_test.std():.4f}")

## 6. Environnement de Hedging

Calcul du PnL avec coûts de transaction.

In [ ]:
class HedgingEnv:
    """Environnement pour calculer le PnL de la stratégie."""
    
    def __init__(self, config: MarketConfig, model: nn.Module, device=DEVICE):
        self.cfg = config
        self.model = model
        self.device = device
        self.dt = config.T / config.n_steps
        
    def compute_pnl(self, data: Dict[str, np.ndarray]) -> torch.Tensor:
        """
        Calcule le PnL de la stratégie de hedging.
        
        Args:
            data: dict avec 'S', 'dS', 'payoff'
        
        Returns:
            PnL (n_paths,)
        """
        S = torch.tensor(data['S'], device=self.device, dtype=DTYPE)
        dS = torch.tensor(data['dS'], device=self.device, dtype=DTYPE)
        payoff = torch.tensor(data['payoff'], device=self.device, dtype=DTYPE)
        
        n_paths = S.shape[0]
        pnl = -payoff  # On part du payoff négatif (on est vendeur)
        
        hedge_prev = torch.zeros(n_paths, device=self.device)
        
        for t in range(self.cfg.n_steps):
            # État actuel
            S_t = S[:, t]
            t_val = torch.full((n_paths,), t * self.dt, device=self.device)
            
            # Ratio de hedging via le modèle
            hedge_t = self.model(S_t, t_val, self.cfg.S0, self.cfg.T)
            
            # PnL du hedge
            pnl += hedge_prev * dS[:, t]
            
            # Coûts de transaction
            trade_amount = torch.abs(hedge_t - hedge_prev)
            transaction_cost = self.cfg.cost_s * trade_amount * S_t
            pnl -= transaction_cost
            
            hedge_prev = hedge_t
        
        # Liquidation finale
        S_T = S[:, -1]
        pnl += hedge_prev * (S_T - S[:, -2])
        pnl -= self.cfg.cost_s * torch.abs(hedge_prev) * S_T
        
        return pnl

# Test de l'environnement
env = HedgingEnv(cfg.market, model, DEVICE)
test_pnl = env.compute_pnl(test_data)
print(f"PnL test: mean={test_pnl.mean():.4f}, std={test_pnl.std():.4f}")

## 7. Training Loop

Entraînement du modèle avec optimisation CVaR.

In [ ]:
class DeepHedgingTrainer:
    """Gestionnaire de training pour Deep Hedging."""
    
    def __init__(self, config: DeepHedgingConfig, model: nn.Module, env: HedgingEnv):
        self.config = config
        self.model = model
        self.env = env
        self.optimizer = optim.Adam(model.parameters(), lr=config.training.lr)
        self.loss_fn = CVaRLoss(alpha=config.training.cvar_alpha)
        self.history = {'train_loss': [], 'val_loss': []}
        
    def train_epoch(self, data: Dict[str, np.ndarray]) -> float:
        """Entraîne le modèle sur une epoch."""
        self.model.train()
        self.optimizer.zero_grad()
        
        # Forward pass
        pnl = self.env.compute_pnl(data)
        
        # Loss
        loss = self.loss_fn(pnl)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def validate(self, data: Dict[str, np.ndarray]) -> Tuple[float, Dict]:
        """Évalue le modèle sur les données de validation."""
        self.model.eval()
        
        with torch.no_grad():
            pnl = self.env.compute_pnl(data)
            loss = self.loss_fn(pnl)
            
            metrics = {
                'mean_pnl': pnl.mean().item(),
                'std_pnl': pnl.std().item(),
                'cvar': loss.item(),
                'var': torch.quantile(-pnl, self.config.training.cvar_alpha).item()
            }
        
        return loss.item(), metrics
    
    def train(self, world):
        """Boucle de training complète."""
        print("\n" + "="*70)
        print("DÉMARRAGE DU TRAINING")
        print("="*70)
        
        # Génération des données
        print("Génération des données...")
        train_data = world.simulate_paths(self.config.market.n_paths_train, seed=self.config.seed)
        val_data = world.simulate_paths(self.config.market.n_paths_val, seed=self.config.seed + 1)
        print(f"Train: {train_data['S'].shape[0]} paths, Val: {val_data['S'].shape[0]} paths")
        
        # Training loop
        for epoch in range(1, self.config.training.n_epochs + 1):
            train_loss = self.train_epoch(train_data)
            self.history['train_loss'].append(train_loss)
            
            if epoch % self.config.training.print_every == 0:
                val_loss, metrics = self.validate(val_data)
                self.history['val_loss'].append(val_loss)
                
                print(f"\nEpoch {epoch}/{self.config.training.n_epochs}")
                print(f"  Train Loss: {train_loss:.6f}")
                print(f"  Val Loss:   {val_loss:.6f}")
                print(f"  PnL: μ={metrics['mean_pnl']:.4f}, σ={metrics['std_pnl']:.4f}")
                print(f"  CVaR={metrics['cvar']:.4f}, VaR={metrics['var']:.4f}")
        
        print("\n" + "="*70)
        print("TRAINING TERMINÉ")
        print("="*70 + "\n")
        
        return self.history

print("Trainer créé et prêt pour l'entraînement.")

## 8. Exécution du Training

Entraînement du modèle de Deep Hedging.

In [ ]:
# Création du monde
world_train = BlackScholesWorld(cfg.market)

# Création du modèle
model = HedgingNetwork(hidden_sizes=[64, 64]).to(DEVICE)

# Création de l'environnement
env = HedgingEnv(cfg.market, model, DEVICE)

# Création du trainer
trainer = DeepHedgingTrainer(cfg, model, env)

# Training
history = trainer.train(world_train)

## 9. Visualisation des Résultats

Analyse des performances du modèle entraîné.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].plot(history['train_loss'], label='Train Loss', alpha=0.7)
if history['val_loss']:
    val_epochs = [i * cfg.training.print_every for i in range(len(history['val_loss']))]
    axes[0].plot(val_epochs, history['val_loss'], label='Val Loss', marker='o', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('CVaR Loss')
axes[0].set_title('Courbe d\'Apprentissage')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution du PnL
model.eval()
test_data = world_train.simulate_paths(10_000, seed=999)
with torch.no_grad():
    test_pnl = env.compute_pnl(test_data).cpu().numpy()

axes[1].hist(test_pnl, bins=50, alpha=0.7, edgecolor='black')
axes[1].axvline(test_pnl.mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {test_pnl.mean():.3f}')
axes[1].axvline(np.percentile(test_pnl, 2.5), color='orange', linestyle='--', linewidth=2, label=f'VaR 2.5%: {np.percentile(test_pnl, 2.5):.3f}')
axes[1].set_xlabel('PnL')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution du PnL (Test)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nStatistiques finales:")
print(f"  PnL moyen: {test_pnl.mean():.4f}")
print(f"  PnL std:   {test_pnl.std():.4f}")
print(f"  VaR 2.5%:  {np.percentile(test_pnl, 2.5):.4f}")
print(f"  CVaR 2.5%: {test_pnl[test_pnl <= np.percentile(test_pnl, 2.5)].mean():.4f}")

## 10. Comparaison avec Delta Hedging Classique

Comparaison de notre stratégie Deep Hedging avec le Delta Hedging théorique.

In [ ]:
from scipy.stats import norm

def black_scholes_delta(S, K, T, r, sigma, t, is_call=True):
    """Calcule le Delta Black-Scholes."""
    tau = T - t
    if tau <= 0:
        return 1.0 if (is_call and S > K) else 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    
    if is_call:
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1.0

def delta_hedging_pnl(data: Dict[str, np.ndarray], config: MarketConfig) -> np.ndarray:
    """Calcule le PnL avec Delta Hedging classique."""
    S = data['S']
    dS = data['dS']
    payoff = data['payoff']
    
    n_paths = S.shape[0]
    pnl = -payoff
    
    hedge_prev = np.zeros(n_paths)
    dt = config.T / config.n_steps
    
    for t in range(config.n_steps):
        S_t = S[:, t]
        t_val = t * dt
        
        # Delta théorique
        hedge_t = black_scholes_delta(S_t, config.K, config.T, config.r, config.sigma, t_val, config.is_call)
        
        # PnL
        pnl += hedge_prev * dS[:, t]
        
        # Coûts
        transaction_cost = config.cost_s * np.abs(hedge_t - hedge_prev) * S_t
        pnl -= transaction_cost
        
        hedge_prev = hedge_t
    
    # Liquidation
    S_T = S[:, -1]
    pnl += hedge_prev * (S_T - S[:, -2])
    pnl -= config.cost_s * np.abs(hedge_prev) * S_T
    
    return pnl

# Comparaison
print("\nComparaison Deep Hedging vs Delta Hedging:")
print("="*60)

test_data_comp = world_train.simulate_paths(50_000, seed=12345)

# Deep Hedging
model.eval()
with torch.no_grad():
    pnl_deep = env.compute_pnl(test_data_comp).cpu().numpy()

# Delta Hedging
pnl_delta = delta_hedging_pnl(test_data_comp, cfg.market)

# Statistiques
print(f"\nDeep Hedging:")
print(f"  Moyenne:  {pnl_deep.mean():.4f}")
print(f"  Std:      {pnl_deep.std():.4f}")
print(f"  VaR 2.5%: {np.percentile(pnl_deep, 2.5):.4f}")
print(f"  CVaR 2.5%:{pnl_deep[pnl_deep <= np.percentile(pnl_deep, 2.5)].mean():.4f}")

print(f"\nDelta Hedging:")
print(f"  Moyenne:  {pnl_delta.mean():.4f}")
print(f"  Std:      {pnl_delta.std():.4f}")
print(f"  VaR 2.5%: {np.percentile(pnl_delta, 2.5):.4f}")
print(f"  CVaR 2.5%:{pnl_delta[pnl_delta <= np.percentile(pnl_delta, 2.5)].mean():.4f}")

# Visualisation comparative
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(pnl_deep, bins=50, alpha=0.6, label='Deep Hedging', edgecolor='black')
plt.hist(pnl_delta, bins=50, alpha=0.6, label='Delta Hedging', edgecolor='black')
plt.xlabel('PnL')
plt.ylabel('Fréquence')
plt.title('Distribution des PnL')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
metrics = ['Moyenne', 'Std', 'VaR 2.5%', 'CVaR 2.5%']
deep_vals = [pnl_deep.mean(), pnl_deep.std(), np.percentile(pnl_deep, 2.5), 
             pnl_deep[pnl_deep <= np.percentile(pnl_deep, 2.5)].mean()]
delta_vals = [pnl_delta.mean(), pnl_delta.std(), np.percentile(pnl_delta, 2.5),
              pnl_delta[pnl_delta <= np.percentile(pnl_delta, 2.5)].mean()]

x = np.arange(len(metrics))
width = 0.35
plt.bar(x - width/2, deep_vals, width, label='Deep Hedging', alpha=0.8)
plt.bar(x + width/2, delta_vals, width, label='Delta Hedging', alpha=0.8)
plt.xlabel('Métrique')
plt.ylabel('Valeur')
plt.title('Comparaison des Métriques')
plt.xticks(x, metrics, rotation=15)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 11. Analyse des Stratégies de Hedging

Visualisation des ratios de hedging le long d'une trajectoire typique.

In [ ]:
# Simulation d'une trajectoire
single_path = world_train.simulate_paths(1, seed=777)
S_path = single_path['S'][0]
t_grid = single_path['t_grid']

# Calcul des hedges
deep_hedges = []
delta_hedges = []

model.eval()
for t_idx in range(cfg.market.n_steps):
    S_t = S_path[t_idx]
    t_val = t_grid[t_idx]
    
    # Deep Hedging
    with torch.no_grad():
        S_tensor = torch.tensor([S_t], device=DEVICE, dtype=DTYPE)
        t_tensor = torch.tensor([t_val], device=DEVICE, dtype=DTYPE)
        hedge_deep = model(S_tensor, t_tensor, cfg.market.S0, cfg.market.T).cpu().item()
    deep_hedges.append(hedge_deep)
    
    # Delta Hedging
    hedge_delta = black_scholes_delta(S_t, cfg.market.K, cfg.market.T, cfg.market.r, 
                                      cfg.market.sigma, t_val, cfg.market.is_call)
    delta_hedges.append(hedge_delta)

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Prix du sous-jacent
axes[0].plot(t_grid, S_path, linewidth=2, color='blue')
axes[0].axhline(cfg.market.K, color='red', linestyle='--', linewidth=1.5, label=f'Strike = {cfg.market.K}')
axes[0].set_ylabel('Prix du Sous-jacent')
axes[0].set_title('Trajectoire du Prix')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Ratios de hedging
axes[1].plot(t_grid[:-1], deep_hedges, label='Deep Hedging', linewidth=2, alpha=0.8)
axes[1].plot(t_grid[:-1], delta_hedges, label='Delta Hedging', linewidth=2, alpha=0.8, linestyle='--')
axes[1].set_xlabel('Temps')
axes[1].set_ylabel('Ratio de Hedging')
axes[1].set_title('Stratégies de Couverture')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Test avec Sauts de Merton (Optionnel)

Entraînement du modèle sur un monde avec sauts pour tester la robustesse.

In [ ]:
# Configuration avec sauts
cfg_merton = DeepHedgingConfig()
cfg_merton.market.use_jumps = True
cfg_merton.market.lambda_jump = 2.0
cfg_merton.market.mu_J = -0.05
cfg_merton.market.sigma_J = 0.15
cfg_merton.market.n_paths_train = 100_000  # Moins de paths pour aller plus vite
cfg_merton.market.n_paths_val = 20_000
cfg_merton.training.n_epochs = 30

print("Configuration Merton:")
print(f"  λ = {cfg_merton.market.lambda_jump}")
print(f"  μ_J = {cfg_merton.market.mu_J}")
print(f"  σ_J = {cfg_merton.market.sigma_J}")

# Création du monde Merton
world_merton = MertonWorld(cfg_merton.market)

# Nouveau modèle
model_merton = HedgingNetwork(hidden_sizes=[64, 64]).to(DEVICE)
env_merton = HedgingEnv(cfg_merton.market, model_merton, DEVICE)
trainer_merton = DeepHedgingTrainer(cfg_merton, model_merton, env_merton)

# Training
print("\nTraining sur Merton Jump-Diffusion...")
history_merton = trainer_merton.train(world_merton)

## 13. Sauvegarde du Modèle

In [ ]:
# Sauvegarde
torch.save({
    'model_state_dict': model.state_dict(),
    'config': cfg,
    'history': history
}, 'deep_hedging_model.pt')

print("Modèle sauvegardé dans 'deep_hedging_model.pt'")

# Pour charger:
# checkpoint = torch.load('deep_hedging_model.pt')
# model.load_state_dict(checkpoint['model_state_dict'])

## Conclusion

Ce notebook a implémenté une stratégie de **Deep Hedging** complète:

✅ Simulation Black-Scholes et Merton optimisée  
✅ Architecture neuronale pour la couverture  
✅ Optimisation CVaR du risque de queue  
✅ Comparaison avec Delta Hedging classique  
✅ Visualisations et analyses détaillées  

### Points clés

- Le Deep Hedging peut surpasser le Delta Hedging en présence de frictions
- L'optimisation CVaR réduit le risque de queue
- L'approche est robuste aux sauts (Merton)

### Extensions possibles

- Tester différentes architectures (LSTM, Attention)
- Ajouter des features additionnelles (volatilité implicite, Greeks)
- Implémenter d'autres fonctions objectif (Sharpe, Sortino)
- Étendre à des options plus complexes (barrières, asiatiques)